# Setup

In [1]:
# Import dependencies
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from splinter import Browser
from bs4 import BeautifulSoup
from flask import Flask, jsonify

In [2]:
# Configure browser for scraping

# Windows
# executable_path = {'executable_path': 'chromedriver.exe'}

# Mac
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}

# All OS
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Define URL variables
coffeeBeanCorralURL = 'https://www.coffeebeancorral.com/categories/Green-Coffee-Beans/All-Coffees.aspx'
freshRoastedCoffeeURL = 'https://www.freshroastedcoffee.com/collections/green-coffee'
amazonURL = 'https://www.amazon.com/Best-Sellers-Grocery-Gourmet-Food-Unroasted-Coffee-Beans/zgbs/grocery/979887011'
deansBeansURL = 'https://deansbeans.com/our-products/roasted-coffees/green-unroasted-coffee-beans.html'

# Scrape Coffee Bean Corral for:
- Name (listing title)
- Price (listing price)
- Star Rating (Number of stars given, out of five.)
- Source (Coffee Bean Corral)
- URL (link to the bean's individual page)

In [4]:
# Launch the browser
browser.visit(coffeeBeanCorralURL)

In [5]:
# Pull in browser content to form the soup
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [6]:
# Narrow the search scope to the product container
results = soup.find_all('div', class_='product-info')
print(results)

[<div class="product-info">
<div class="SingleProductDisplayName recordname">
<a href="/product/Bali-Hai-Organic-Blend-SWP-Decaf.aspx" id="ctl00_MainContentHolder_ucDevelisysFacetedSearchProductGrid_ucProductGridDisplay_rpProductGrid_ctl01_SingleProductDisplay_NameHyperLink">Bali Hai Organic Decaf Indonesia Blend SWP</a>
</div>
<div id="ctl00_MainContentHolder_ucDevelisysFacetedSearchProductGrid_ucProductGridDisplay_rpProductGrid_ctl01_SingleProductDisplay_SingleProductDisplayRating">
<img alt="5 stars" class="recordrating" id="ctl00_MainContentHolder_ucDevelisysFacetedSearchProductGrid_ucProductGridDisplay_rpProductGrid_ctl01_SingleProductDisplay_RatingStars" src="/BVModules/Themes/CoffeeBean/images/buttons/Stars5.svg" style="border-width:0px;"/>
<a class="recordreviewcount" href="/product/Bali-Hai-Organic-Blend-SWP-Decaf.aspx#Write" id="ctl00_MainContentHolder_ucDevelisysFacetedSearchProductGrid_ucProductGridDisplay_rpProductGrid_ctl01_SingleProductDisplay_ReviewCount">20 reviews</a>

In [7]:
# Loop through all of the results to find the correct data
names = []
prices = []
ratings = []
URLs = []
sources = []


for result in results:
    # Name
    try:
        result_name = result.find('a').text
        print(result_name)
        names.append(result_name)
    except:
        print("Name failed")
        names.append("Failed")
        
    # Prices
    try:
        result_price = result.find('span', class_='PriceLabel').text
        print(result_price)
        prices.append(result_price)
        
    except:
        print("Price failed")
        prices.append("Failed")
        
    # Ratings
    try:
        result_rating = result.find('img')["alt"]
        print(result_rating)
        ratings.append(result_rating)
    
    except:
        print("Rating failed")
        ratings.append("Failed")
        
    # URL
    try:
        baseURL = 'https://www.coffeebeancorral.com'
        result_URL = result.find('a')
        completeURL = baseURL + result_URL["href"]
        print(completeURL)
        URLs.append(completeURL)
    
    except:
        print("URL failed")
        URLs.append("Failed")
        
    # Source
    try:
        sources.append("Coffee Bean Corral")
    
    except:
        print("Source failed")
        sources.append("Failed")

Bali Hai Organic Decaf Indonesia Blend SWP
$7.00
5 stars
https://www.coffeebeancorral.com/product/Bali-Hai-Organic-Blend-SWP-Decaf.aspx
Bali Organic Blue Moon
$7.75
5 stars
https://www.coffeebeancorral.com/product/Bali-Organic-Blue-Moon__BALIBEANS.aspx
Big Black Organic CBC Blend
$6.50
5 stars
https://www.coffeebeancorral.com/product/Big-Black-Organic-CBC-Blend__BIGBLACKORG.aspx
Brazil Cerrado Natural 17/18
$4.00
4 stars
https://www.coffeebeancorral.com/product/Brazil-Cerrado-Natural-1718__BRACERRADO.aspx
Brazil Daterra CHC Reserve Espresso
$7.25
5 stars
https://www.coffeebeancorral.com/product/Daterra-Espresso-Blend.aspx
Brazil Daterra Pearl Bourbon Peaberry RFA
$8.50
5 stars
https://www.coffeebeancorral.com/product/Brazil-Bourbon-Peaberry-Daterra-Estate-Pearl-Bourbon__DATPEARLBBN.aspx
Brazil Daterra Sweet Blue RFA
$8.50
5 stars
https://www.coffeebeancorral.com/product/Brazil-Sweet-Blue-Daterra-Estate__DATBLUE.aspx
Brazil Daterra Sweet Yellow RFA
$8.50
5 stars
https://www.coffeebeanco

In [8]:
# Pull lists into a data frame
coffeeCorralData = {'Name': names, 
                   'Price': prices,
                   'Rating': ratings,
                   'URL': URLs,
                   'Source': sources}
coffeeCorral = pd.DataFrame(data=coffeeCorralData)
coffeeCorral.head()

,Name,Price,Rating,URL,Source
0,Bali Hai Organic Decaf Indonesia Blend SWP,$7.00,5 stars,https://www.coffeebeancorral.com/product/Bali-...,Coffee Bean Corral
1,Bali Organic Blue Moon,$7.75,5 stars,https://www.coffeebeancorral.com/product/Bali-...,Coffee Bean Corral
2,Big Black Organic CBC Blend,$6.50,5 stars,https://www.coffeebeancorral.com/product/Big-B...,Coffee Bean Corral
3,Brazil Cerrado Natural 17/18,$4.00,4 stars,https://www.coffeebeancorral.com/product/Brazi...,Coffee Bean Corral
4,Brazil Daterra CHC Reserve Espresso,$7.25,5 stars,https://www.coffeebeancorral.com/product/Dater...,Coffee Bean Corral


In [9]:
# Remove dollar sign from Price
coffeeCorral['Price'] = coffeeCorral['Price'].str.replace('$', '')
coffeeCorral.head()

,Name,Price,Rating,URL,Source
0,Bali Hai Organic Decaf Indonesia Blend SWP,7.00,5 stars,https://www.coffeebeancorral.com/product/Bali-...,Coffee Bean Corral
1,Bali Organic Blue Moon,7.75,5 stars,https://www.coffeebeancorral.com/product/Bali-...,Coffee Bean Corral
2,Big Black Organic CBC Blend,6.50,5 stars,https://www.coffeebeancorral.com/product/Big-B...,Coffee Bean Corral
3,Brazil Cerrado Natural 17/18,4.00,4 stars,https://www.coffeebeancorral.com/product/Brazi...,Coffee Bean Corral
4,Brazil Daterra CHC Reserve Espresso,7.25,5 stars,https://www.coffeebeancorral.com/product/Dater...,Coffee Bean Corral


In [10]:
# Remove extra words from the Rating column
coffeeCorral['Rating'] = [x.split(' ')[0] for x in coffeeCorral['Rating']]
coffeeCorral.head()

,Name,Price,Rating,URL,Source
0,Bali Hai Organic Decaf Indonesia Blend SWP,7.00,5,https://www.coffeebeancorral.com/product/Bali-...,Coffee Bean Corral
1,Bali Organic Blue Moon,7.75,5,https://www.coffeebeancorral.com/product/Bali-...,Coffee Bean Corral
2,Big Black Organic CBC Blend,6.50,5,https://www.coffeebeancorral.com/product/Big-B...,Coffee Bean Corral
3,Brazil Cerrado Natural 17/18,4.00,4,https://www.coffeebeancorral.com/product/Brazi...,Coffee Bean Corral
4,Brazil Daterra CHC Reserve Espresso,7.25,5,https://www.coffeebeancorral.com/product/Dater...,Coffee Bean Corral


# Scrape Amazon for:
- Name (listing title)
- Price (listing price)
- Star Rating (Number of stars given, out of five.)
- Source (Amazon)
- URL (link to the bean's individual page)

In [11]:
# Launch the browser
browser.visit(amazonURL)

In [12]:
# Pull in browser content to form the soup
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [13]:
# Narrow the search scope to the product container
results = soup.find_all('span', class_='aok-inline-block zg-item')
print(results)

[<span class="aok-inline-block zg-item"><a class="a-link-normal" href="/Single-Unroasted-Coffee-Specialty-Nicaraguan/dp/B014THN4OI/ref=zg_bs_979887011_1/138-7169295-4669266?_encoding=UTF8&amp;psc=1&amp;refRID=3P9NWFJSJMVXEZZT2YYD"><span class="zg-text-center-align"><div class="a-section a-spacing-small"><img alt="3 Lbs, Single Origin Unroasted Green Coffee Beans, Specialty Grade From Single Nicaraguan Estate, Direct Trade" height="200" src="https://images-na.ssl-images-amazon.com/images/I/913XLW8BLKL._AC_UL200_SR200,200_.jpg" width="200"/></div></span>
<div aria-hidden="true" class="p13n-sc-truncated" data-rows="2" title="3 Lbs, Single Origin Unroasted Green Coffee Beans, Specialty Grade From Single Nicaraguan Estate, Direct Trade">3 Lbs, Single Origin Unroasted Green Coffee Beans, Specialty Grade From Single Nicaraguan…</div>
</a>
<div class="a-icon-row a-spacing-none">
<a class="a-link-normal" href="/product-reviews/B014THN4OI/ref=zg_bs_979887011_cr_1/138-7169295-4669266?ie=UTF8&amp;

In [14]:
# Loop through all of the results to find the correct data
names = []
prices = []
ratings = []
URLs = []
sources = []


for result in results:
    # Name
    try:
        resultsLevel1 = result.find('a', class_='a-link-normal')
        resultsLevel2 = resultsLevel1.find('span', class_='zg-text-center-align')
        resultsLevel3 = resultsLevel2.find('div', class_='a-section a-spacing-small')
        resultsLevel4 = resultsLevel3.find('img')
        results_name = resultsLevel4["alt"]
        print(results_name)
        names.append(results_name)
    except:
        print("Name failed")
        names.append("Failed")
        
    # Price
    try:
        priceResults = result.find('span', class_='p13n-sc-price').text
        print(priceResults)
        prices.append(priceResults)  
    except:
        print("Price failed")
        prices.append("Failed")
        
    # URL
    baseURL = 'https://amazon.com'
    try:
        resultsLink = result.find('a', class_='a-link-normal')
        resultsLink = resultsLevel1['href']
        resultsLink = baseURL + resultsLink
        print(resultsLink)
        URLs.append(resultsLink)
    except:
        print("Failed")
        URLs.append("Failed")
        
    # Source
    try:
        sources.append("Amazon Grocery")
    
    except:
        print("Source failed")
        sources.append("Failed")
        
    # Ratings
    try:
        resultsLevel1 = result.find('div', class_='a-icon-row a-spacing-none')
        result_rating = resultsLevel1.find('a')["title"]
        print(result_rating)
        ratings.append(result_rating)
    
    except:
        print("Rating failed")
        ratings.append("Failed")

3 Lbs, Single Origin Unroasted Green Coffee Beans, Specialty Grade From Single Nicaraguan Estate, Direct Trade
$19.99
https://amazon.com/Single-Unroasted-Coffee-Specialty-Nicaraguan/dp/B014THN4OI/ref=zg_bs_979887011_1/138-7169295-4669266?_encoding=UTF8&psc=1&refRID=3P9NWFJSJMVXEZZT2YYD
4.5 out of 5 stars
COLOMBIAN SUPREMO Unroasted Green Raw Coffee Beans, 5 LB Bulk Bag, 100% Arabica Top Grade Extra Large
$29.79
https://amazon.com/COLOMBIAN-SUPREMO-Unroasted-Coffee-Arabica/dp/B0792L8BDV/ref=zg_bs_979887011_2/138-7169295-4669266?_encoding=UTF8&psc=1&refRID=3P9NWFJSJMVXEZZT2YYD
4.4 out of 5 stars
Unroasted Green Coffee Bean Sampler Pack - 4LBS - 100% raw arabica coffee beans - COLOMBIA, ETHIOPIA, GUATEMALA, COSTA RICA, BRAZIL
$34.99
https://amazon.com/Unroasted-Green-Coffee-Bean-Sampler/dp/B07RMKVJ24/ref=zg_bs_979887011_3/138-7169295-4669266?_encoding=UTF8&psc=1&refRID=3P9NWFJSJMVXEZZT2YYD
5.0 out of 5 stars
3 Lb, Single Origin Unroasted Green Coffee Beans, Specialty Grade From Single Nic

In [15]:
# Pull lists into a data frame
amazonData = {'Name': names, 
                   'Price': prices,
                   'Rating': ratings,
                   'URL': URLs,
                   'Source': sources}
amazon = pd.DataFrame(data=amazonData)
amazon.head()

,Name,Price,Rating,URL,Source
0,"3 Lbs, Single Origin Unroasted Green Coffee Be...",$19.99,4.5 out of 5 stars,https://amazon.com/Single-Unroasted-Coffee-Spe...,Amazon Grocery
1,COLOMBIAN SUPREMO Unroasted Green Raw Coffee B...,$29.79,4.4 out of 5 stars,https://amazon.com/COLOMBIAN-SUPREMO-Unroasted...,Amazon Grocery
2,Unroasted Green Coffee Bean Sampler Pack - 4LB...,$34.99,5.0 out of 5 stars,https://amazon.com/Unroasted-Green-Coffee-Bean...,Amazon Grocery
3,"3 Lb, Single Origin Unroasted Green Coffee Bea...",$21.49,4.5 out of 5 stars,https://amazon.com/Unroasted-Coffee-Specialty-...,Amazon Grocery
4,Ethiopian Yirgacheffe Green Unroasted Coffee B...,$12.97,4.1 out of 5 stars,https://amazon.com/Ethiopian-Yirgacheffe-Green...,Amazon Grocery


In [16]:
# Remove dollar sign from Price
amazon['Price'] = amazon['Price'].str.replace('$', '')
amazon.head()

,Name,Price,Rating,URL,Source
0,"3 Lbs, Single Origin Unroasted Green Coffee Be...",19.99,4.5 out of 5 stars,https://amazon.com/Single-Unroasted-Coffee-Spe...,Amazon Grocery
1,COLOMBIAN SUPREMO Unroasted Green Raw Coffee B...,29.79,4.4 out of 5 stars,https://amazon.com/COLOMBIAN-SUPREMO-Unroasted...,Amazon Grocery
2,Unroasted Green Coffee Bean Sampler Pack - 4LB...,34.99,5.0 out of 5 stars,https://amazon.com/Unroasted-Green-Coffee-Bean...,Amazon Grocery
3,"3 Lb, Single Origin Unroasted Green Coffee Bea...",21.49,4.5 out of 5 stars,https://amazon.com/Unroasted-Coffee-Specialty-...,Amazon Grocery
4,Ethiopian Yirgacheffe Green Unroasted Coffee B...,12.97,4.1 out of 5 stars,https://amazon.com/Ethiopian-Yirgacheffe-Green...,Amazon Grocery


In [17]:
# Remove extra words from the Rating column
amazon['Rating'] = [x.split(' ')[0] for x in amazon['Rating']]
amazon.head()

,Name,Price,Rating,URL,Source
0,"3 Lbs, Single Origin Unroasted Green Coffee Be...",19.99,4.5,https://amazon.com/Single-Unroasted-Coffee-Spe...,Amazon Grocery
1,COLOMBIAN SUPREMO Unroasted Green Raw Coffee B...,29.79,4.4,https://amazon.com/COLOMBIAN-SUPREMO-Unroasted...,Amazon Grocery
2,Unroasted Green Coffee Bean Sampler Pack - 4LB...,34.99,5.0,https://amazon.com/Unroasted-Green-Coffee-Bean...,Amazon Grocery
3,"3 Lb, Single Origin Unroasted Green Coffee Bea...",21.49,4.5,https://amazon.com/Unroasted-Coffee-Specialty-...,Amazon Grocery
4,Ethiopian Yirgacheffe Green Unroasted Coffee B...,12.97,4.1,https://amazon.com/Ethiopian-Yirgacheffe-Green...,Amazon Grocery


# Scrape Dean's Beans for:
- Name (listing title)
- Price (listing price)
- Star Rating (Number of stars given, out of five.)
- Source (Amazon)
- URL (link to the bean's individual page)

In [18]:
# Launch the browser
browser.visit(deansBeansURL)

In [19]:
# Pull in browser content to form the soup
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [20]:
# Narrow the search scope to the product container
results = soup.find_all('div', class_='item-content')
print(results)

[<div class="item-content"><h2 class="item-name"><a href="https://deansbeans.com/our-products/roasted-coffees/green-unroasted-coffee-beans/kraft-paper-bags-with-tin-tie.html" title="10 Pack of Kraft Paper Bags (with Tin Tie)">10 Pack of Kraft Paper Bags (with Tin Tie)</a></h2><div class="item-desc"><a href="https://deansbeans.com/our-products/roasted-coffees/green-unroasted-coffee-beans/kraft-paper-bags-with-tin-tie.html" title="10 Pack of Kraft Paper Bags (with Tin Tie)"><p>10 bags for $2.50</p></a></div><div class="item-link"> <button class="buy-link btn-go" data-id="3236">Buy now</button></div></div>, <div class="item-content"><h2 class="item-name"><a href="https://deansbeans.com/our-products/roasted-coffees/green-unroasted-coffee-beans/green-bean-unroasted-coffee-sampler.html" title="Green Bean (Unroasted) Sampler Pack">Green Bean (Unroasted) Sampler Pack</a></h2><div class="item-desc"><a href="https://deansbeans.com/our-products/roasted-coffees/green-unroasted-coffee-beans/green-b

In [21]:
# Loop through all of the results to find the correct data
names = []
prices = []
ratings = []
URLs = []
sources = []


for result in results:
    # Name
    try:
        resultsLevel1 = result.find('h2', class_='item-name')
        resultsLevel2 = resultsLevel1.find('a')
        results_name = resultsLevel2["title"]
        print(results_name)
        names.append(results_name)
    except:
        print("Name failed")
        names.append("Failed")
        
    # Price
    try:
        resultsLevel1 = result.find('div', class_='item-desc')
        resultsLevel2 = resultsLevel1.find('a')
        resultsLevel3 = resultsLevel2.find('h5')
        results_price = resultsLevel3.text
        print(results_price)
        prices.append(results_price)
    except:
        print("Price failed")
        prices.append("Failed")
        
    # URL
    try:
        resultsLevel1 = result.find('div', class_='item-desc')
        resultsLevel2 = resultsLevel1.find('a')
        results_URL = resultsLevel2["href"]
        print(results_URL)
        URLs.append(results_URL)
    except:
        print("Failed")
        URLs.append("Failed")
        
    # Source
    try:
        sources.append("Dean's Beans")
    
    except:
        print("Source failed")
        sources.append("Failed")
        
    # Ratings
    try:
        result_rating = result.find('svg')["aria-label"]
        print(result_rating)
        ratings.append(result_rating)
    
    except:
        print("Rating failed")
        ratings.append("Failed")

10 Pack of Kraft Paper Bags (with Tin Tie)
Price failed
https://deansbeans.com/our-products/roasted-coffees/green-unroasted-coffee-beans/kraft-paper-bags-with-tin-tie.html
Rating failed
Green Bean (Unroasted) Sampler Pack
Six 8-oz bags of assorted
https://deansbeans.com/our-products/roasted-coffees/green-unroasted-coffee-beans/green-bean-unroasted-coffee-sampler.html
Rating failed
Home Roasting Kit
$189.99
https://deansbeans.com/our-products/roasted-coffees/green-unroasted-coffee-beans/home-roasting-green-bean-sampler-kit.html
Rating failed
Organic Colombian Green Coffee (Unroasted)
1lb - $5.50
https://deansbeans.com/our-products/roasted-coffees/green-unroasted-coffee-beans/organic-colombian-green-coffee-unroasted.html
5 out of 5 stars
Organic Ethiopian Green Coffee (Unroasted)
1lb - $5.85
https://deansbeans.com/our-products/roasted-coffees/green-unroasted-coffee-beans/organic-ethiopian-yirgacheffe-green-coffee-unroasted.html
5 out of 5 stars
Organic Green Espresso - Smooth (Unroasted)

In [22]:
# Pull lists into a data frame
deansBeansData = {'Name': names, 
                   'Price': prices,
                   'Rating': ratings,
                   'URL': URLs,
                   'Source': sources}
deansBeans = pd.DataFrame(data=deansBeansData)
deansBeans.head()

,Name,Price,Rating,URL,Source
0,10 Pack of Kraft Paper Bags (with Tin Tie),Failed,Failed,https://deansbeans.com/our-products/roasted-co...,Dean's Beans
1,Green Bean (Unroasted) Sampler Pack,Six 8-oz bags of assorted,Failed,https://deansbeans.com/our-products/roasted-co...,Dean's Beans
2,Home Roasting Kit,$189.99,Failed,https://deansbeans.com/our-products/roasted-co...,Dean's Beans
3,Organic Colombian Green Coffee (Unroasted),1lb - $5.50,5 out of 5 stars,https://deansbeans.com/our-products/roasted-co...,Dean's Beans
4,Organic Ethiopian Green Coffee (Unroasted),1lb - $5.85,5 out of 5 stars,https://deansbeans.com/our-products/roasted-co...,Dean's Beans


In [23]:
# Remove extra words from Price
Entry = []
deansBeans['Price'] = deansBeans['Price'].str.rsplit()
for i in deansBeans['Price']:
    finalEntry = len(i)-1
    Entry.append(i[finalEntry])
deansBeans['Price'] = Entry
deansBeans.head()

,Name,Price,Rating,URL,Source
0,10 Pack of Kraft Paper Bags (with Tin Tie),Failed,Failed,https://deansbeans.com/our-products/roasted-co...,Dean's Beans
1,Green Bean (Unroasted) Sampler Pack,assorted,Failed,https://deansbeans.com/our-products/roasted-co...,Dean's Beans
2,Home Roasting Kit,$189.99,Failed,https://deansbeans.com/our-products/roasted-co...,Dean's Beans
3,Organic Colombian Green Coffee (Unroasted),$5.50,5 out of 5 stars,https://deansbeans.com/our-products/roasted-co...,Dean's Beans
4,Organic Ethiopian Green Coffee (Unroasted),$5.85,5 out of 5 stars,https://deansbeans.com/our-products/roasted-co...,Dean's Beans


In [24]:
deansBeans['Price'] = Entry

In [25]:
# Remove dollar sign from Price
deansBeans['Price'] = deansBeans['Price'].str.replace('$', '')
deansBeans.head()

,Name,Price,Rating,URL,Source
0,10 Pack of Kraft Paper Bags (with Tin Tie),Failed,Failed,https://deansbeans.com/our-products/roasted-co...,Dean's Beans
1,Green Bean (Unroasted) Sampler Pack,assorted,Failed,https://deansbeans.com/our-products/roasted-co...,Dean's Beans
2,Home Roasting Kit,189.99,Failed,https://deansbeans.com/our-products/roasted-co...,Dean's Beans
3,Organic Colombian Green Coffee (Unroasted),5.50,5 out of 5 stars,https://deansbeans.com/our-products/roasted-co...,Dean's Beans
4,Organic Ethiopian Green Coffee (Unroasted),5.85,5 out of 5 stars,https://deansbeans.com/our-products/roasted-co...,Dean's Beans


In [26]:
# Remove extra words from the Rating column
deansBeans['Rating'] = [x.split(' ')[0] for x in deansBeans['Rating']]
deansBeans.head()

,Name,Price,Rating,URL,Source
0,10 Pack of Kraft Paper Bags (with Tin Tie),Failed,Failed,https://deansbeans.com/our-products/roasted-co...,Dean's Beans
1,Green Bean (Unroasted) Sampler Pack,assorted,Failed,https://deansbeans.com/our-products/roasted-co...,Dean's Beans
2,Home Roasting Kit,189.99,Failed,https://deansbeans.com/our-products/roasted-co...,Dean's Beans
3,Organic Colombian Green Coffee (Unroasted),5.50,5,https://deansbeans.com/our-products/roasted-co...,Dean's Beans
4,Organic Ethiopian Green Coffee (Unroasted),5.85,5,https://deansbeans.com/our-products/roasted-co...,Dean's Beans


# Scrape Fresh Roasted Coffee for:
- Name (listing title)
- Price (listing price)
- Star Rating (Number of stars given, out of five.)
- Source (Amazon)
- URL (link to the bean's individual page)

In [27]:
# Launch the browser
browser.visit(freshRoastedCoffeeURL)

In [28]:
# Pull in browser content to form the soup
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [29]:
# Narrow the search scope to the product container
resultFreshLayer1 = soup.find_all('div', class_='bc-sf-filter-product-item-inner')
resultFreshLayer2 = soup.find_all('div', class_='bc-sf-filter-product-bottom')
resultFreshLayer3 = soup.find_all('span', class_='yotpo-stars')
resultFreshLayer4 = soup.find_all('p', class_='bc-sf-filter-product-item-price')

In [30]:
# URL
URLs = []

for result in resultFreshLayer1:
    try:
        baseURL = 'https://www.freshroastedcoffee.com'
        result_URL = result.find('a')
        completeURL = baseURL + result_URL["href"]
        print(completeURL)
        URLs.append(completeURL)
    except:
        print("URL failed")
        URLs.append("Failed")


https://www.freshroastedcoffee.com/collections/green-coffee/products/colombian-fto-green-coffee
https://www.freshroastedcoffee.com/collections/green-coffee/products/guatemala-huehuetenango-fto-green-coffee
https://www.freshroastedcoffee.com/collections/green-coffee/products/organic-bali-blue-moon-rfa-green-coffee
https://www.freshroastedcoffee.com/collections/green-coffee/products/costa-rica-tarrazu-green-coffee
https://www.freshroastedcoffee.com/collections/green-coffee/products/organic-mexican-chiapas-green-coffee
https://www.freshroastedcoffee.com/collections/green-coffee/products/sumatra-fto-green-coffee
https://www.freshroastedcoffee.com/collections/green-coffee/products/organic-java-taman-dadar-rfa-green-coffee
https://www.freshroastedcoffee.com/collections/green-coffee/products/ethiopian-yirgacheffe-green-coffee
https://www.freshroastedcoffee.com/collections/green-coffee/products/ethiopian-sidamo-fto-green-coffee
https://www.freshroastedcoffee.com/collections/green-coffee/produc

In [31]:
# Name and source
names = []
sources = []

for result in resultFreshLayer2:
    # Name
    try:
        result_name = result.find('p').text
        print(result_name)
        names.append(result_name)
    
    except:
        print("Name failed")
        names.append("Failed")
        
    # Source
    try:
        result_source = result.find('p', class_='bc-sf-filter-product-item-vendor').text
        print(result_source)
        sources.append(result_source)
    
    except:
        print("Source failed")
        sources.append("Failed")

Unroasted Organic Colombian Sierra Nevada Green Coffee - Fair Trade
Fresh Roasted Coffee
Unroasted Organic Guatemala Huehuetenango Green Coffee - Fair Trade
Fresh Roasted Coffee
Unroasted Organic Bali Blue Moon Green Coffee - RFA
Fresh Roasted Coffee
Unroasted Costa Rica Tarrazu Green Coffee
Fresh Roasted Coffee
Unroasted Organic Mexican Chiapas Green Coffee
Fresh Roasted Coffee
Unroasted Organic Sumatra Green Coffee - Fair Trade, RFA
Fresh Roasted Coffee
Unroasted Organic Java Taman Dadar Green Coffee - RFA
Fresh Roasted Coffee
Unroasted Ethiopian Yirgacheffe Kochere Green Coffee
Fresh Roasted Coffee
Unroasted Organic Ethiopian Sidamo Green Coffee - Fair Trade
Fresh Roasted Coffee
Unroasted Colombian Supremo Green Coffee
Fresh Roasted Coffee
Unroasted Organic Ethiopian Yirgacheffe Green Coffee - Fair Trade
Fresh Roasted Coffee
Unroasted 100% Jamaica Blue Mountain Green Coffee - Direct Trade
Fresh Roasted Coffee
Unroasted Kenya AA Green Coffee
Fresh Roasted Coffee
Unroasted Sumatra Man

In [32]:
# Prices
prices = []
for result in resultFreshLayer4:
    try: 
        result_price = result.find('span', class_='bc-sf-filter-product-item-regular-price').text
        print(result_price)
        prices.append(result_price)
    
    except:
        print("Price failed")
        prices.append("Failed")

$8.95
$7.95
$10.95
$6.95
$6.95
$8.50
$6.95
$7.95
$7.95
$6.95
$8.95
$29.95
$8.95
$7.95
$7.95
$8.95
$14.95
$5.95


In [33]:
# Ratings
ratings = []
for result in resultFreshLayer3:
    try: 
        result_rating = result.find('span', class_='sr-only').text
        print(result_rating)
        ratings.append(result_rating)
    
    except:
        print("Rating failed")
        ratings.append("Failed")

4.8 star rating
5.0 star rating
4.7 star rating
4.7 star rating
4.8 star rating
5.0 star rating
4.6 star rating
4.8 star rating
4.9 star rating
4.8 star rating
4.9 star rating
4.8 star rating
5.0 star rating
4.9 star rating
5.0 star rating
4.7 star rating
4.9 star rating
4.8 star rating
4.7 star rating


In [34]:
# Remove the last item in the ratings list (due to incorrect info)
ratings.pop()
print(ratings)

['4.8 star rating', '5.0 star rating', '4.7 star rating', '4.7 star rating', '4.8 star rating', '5.0 star rating', '4.6 star rating', '4.8 star rating', '4.9 star rating', '4.8 star rating', '4.9 star rating', '4.8 star rating', '5.0 star rating', '4.9 star rating', '5.0 star rating', '4.7 star rating', '4.9 star rating', '4.8 star rating']


In [35]:
# Pull lists into a data frame
freshRoastData = {'Name': names, 
                   'Price': prices,
                   'Rating': ratings,
                   'URL': URLs,
                   'Source': sources}
freshRoast = pd.DataFrame(data=freshRoastData)
freshRoast.head()

,Name,Price,Rating,URL,Source
0,Unroasted Organic Colombian Sierra Nevada Gree...,$8.95,4.8 star rating,https://www.freshroastedcoffee.com/collections...,Fresh Roasted Coffee
1,Unroasted Organic Guatemala Huehuetenango Gree...,$7.95,5.0 star rating,https://www.freshroastedcoffee.com/collections...,Fresh Roasted Coffee
2,Unroasted Organic Bali Blue Moon Green Coffee ...,$10.95,4.7 star rating,https://www.freshroastedcoffee.com/collections...,Fresh Roasted Coffee
3,Unroasted Costa Rica Tarrazu Green Coffee,$6.95,4.7 star rating,https://www.freshroastedcoffee.com/collections...,Fresh Roasted Coffee
4,Unroasted Organic Mexican Chiapas Green Coffee,$6.95,4.8 star rating,https://www.freshroastedcoffee.com/collections...,Fresh Roasted Coffee


In [36]:
# Remove the dollar sign in Price
freshRoast['Price'] = freshRoast['Price'].str.replace('$', '')

# Remove extra words in Rating
freshRoast['Rating'] = freshRoast['Rating'].str.replace('star rating', '')

freshRoast.head()

,Name,Price,Rating,URL,Source
0,Unroasted Organic Colombian Sierra Nevada Gree...,8.95,4.8,https://www.freshroastedcoffee.com/collections...,Fresh Roasted Coffee
1,Unroasted Organic Guatemala Huehuetenango Gree...,7.95,5.0,https://www.freshroastedcoffee.com/collections...,Fresh Roasted Coffee
2,Unroasted Organic Bali Blue Moon Green Coffee ...,10.95,4.7,https://www.freshroastedcoffee.com/collections...,Fresh Roasted Coffee
3,Unroasted Costa Rica Tarrazu Green Coffee,6.95,4.7,https://www.freshroastedcoffee.com/collections...,Fresh Roasted Coffee
4,Unroasted Organic Mexican Chiapas Green Coffee,6.95,4.8,https://www.freshroastedcoffee.com/collections...,Fresh Roasted Coffee


# Drop any rows which contain "Failed"

In [37]:
deansBeans = deansBeans[~deansBeans.Price.str.contains("Failed")]
deansBeans = deansBeans[~deansBeans.Rating.str.contains("Failed")]

In [38]:
coffeeCorral = coffeeCorral[~coffeeCorral.Price.str.contains("Failed")]
coffeeCorral = coffeeCorral[~coffeeCorral.Rating.str.contains("Failed")]

In [39]:
amazon = amazon[~amazon.Price.str.contains("Failed")]
amazon = amazon[~amazon.Rating.str.contains("Failed")]

In [40]:
freshRoast = freshRoast[~freshRoast.Price.str.contains("Failed")]
freshRoast = freshRoast[~freshRoast.Rating.str.contains("Failed")]

# Append 4 dataframes

In [51]:
dfFinal = deansBeans.append(coffeeCorral)
dfFinal = dfFinal.append(amazon)
dfFinal = dfFinal.append(freshRoast)
dfFinal.head(50)

,Name,Price,Rating,URL,Source
3,Organic Colombian Green Coffee (Unroasted),5.50,5,https://deansbeans.com/our-products/roasted-co...,Dean's Beans
4,Organic Ethiopian Green Coffee (Unroasted),5.85,5,https://deansbeans.com/our-products/roasted-co...,Dean's Beans
9,Organic Mexican Green Coffee (Unroasted),5.50,5,https://deansbeans.com/our-products/roasted-co...,Dean's Beans
10,Organic Nicaraguan Green Coffee (Unroasted),5.50,5,https://deansbeans.com/our-products/roasted-co...,Dean's Beans
11,Organic Papua New Guinea Green Coffee (Unroasted),5.85,5,https://deansbeans.com/our-products/roasted-co...,Dean's Beans
0,Bali Hai Organic Decaf Indonesia Blend SWP,7.00,5,https://www.coffeebeancorral.com/product/Bali-...,Coffee Bean Corral
1,Bali Organic Blue Moon,7.75,5,https://www.coffeebeancorral.com/product/Bali-...,Coffee Bean Corral
2,Big Black Organic CBC Blend,6.50,5,https://www.coffeebeancorral.com/product/Big-B...,Coffee Bean Corral
3,Brazil Cerrado Natural 17/18,4.00,4,https://www.coffeebeancorral.com/product/Brazi...,Coffee Bean Corral
4,Brazil Daterra CHC Reserve Espresso,7.25,5,https://www.coffeebeancorral.com/product/Dater...,Coffee Bean Corral


### Create database connection

In [ ]:
# Configure the database connection
rds_connection_string = "postgres:PutYourPassword@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
# Confirm the existing tables
engine.table_names()

# Combine Dataframes
Combine each of the sites' individual data frames into a single, main data frame

# Push the main data frame into SQL

In [ ]:
main_df.to_sql(name='coffee', con=engine, if_exists='replace', index=False)

# Confirm data has been added

In [ ]:
# Read the table data from SQL
pd.read_sql_query('select * from coffee', con=engine).head()